# RSS Testing

Compare 3 methods of RSS feed collection:
- Current MediaCloud API https://github.com/berkmancenter/mediacloud/blob/master/doc/api_2_0_spec/api_2_0_spec.md
- Python feed_seeker library https://github.com/mitmedialab/feed_seeker
- 

In [1]:
from feed_seeker import find_feed_url

In [2]:
find_feed_url('http://www.rightwingwatch.org')

'http://www.rightwingwatch.org/feed/'

## Setup MC API

In [4]:
import mediacloud, json, datetime
mc = mediacloud.api.MediaCloud('7e5510da993cd51097818a48374dff44495cb251f859ec01d61aaae59284fb6c')

In [11]:
mc.feedList(1747)

[{'feed_status': 'active',
  'feed_type': 'syndicated',
  'feeds_id': 28219,
  'last_attempted_download_time': '2018-04-18 06:05:00.525946-04:00',
  'last_new_story_time': '2018-04-12 22:27:48.047295-04:00',
  'last_successful_download_time': '2018-04-18 08:48:13-04:00',
  'media_id': 1747,
  'name': 'Alex Brummer | Mail Online',
  'url': 'http://www.dailymail.co.uk/news/columnist-1001421/alex-brummer.rss'},
 {'feed_status': 'inactive',
  'feed_type': 'syndicated',
  'feeds_id': 28220,
  'last_attempted_download_time': '2016-12-18 05:05:50.831552-05:00',
  'last_new_story_time': '2015-07-17 11:57:30.320253-04:00',
  'last_successful_download_time': '2016-12-18 05:27:09-05:00',
  'media_id': 1747,
  'name': 'Alexandra Shulman | Mail Online',
  'url': 'http://www.dailymail.co.uk/femail/columnist-1005042/alexandra-shulman.rss'},
 {'feed_status': 'inactive',
  'feed_type': 'syndicated',
  'feeds_id': 28221,
  'last_attempted_download_time': '2016-12-18 05:05:50.831552-05:00',
  'last_new_s